In [ ]:
import pandas as pd
import numpy as np
import neattext.functions as nfx
import seaborn as sn

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel


In [ ]:
df = pd.read_csv('udemy_course_data.csv')
df.head()

In [ ]:
dir(nfx)


In [ ]:
df['Clean_title'] = df['course_title'].apply(nfx.remove_stopwords)
df['Clean_title'] = df['Clean_title'].apply(nfx.remove_special_characters)
df['Clean_title'] = df['Clean_title'].str.lower()

In [ ]:
countvect = CountVectorizer()
cv_mat = countvect.fit_transform(df['Clean_title'])


In [ ]:
cosine_sim_mat = cosine_similarity(cv_mat)


In [ ]:
course_index = pd.Series(df.index, index=df['course_title']).drop_duplicates()


In [ ]:
course_dict = dict(course_index)

In [ ]:
# Initialize the variables
keyword = 'Java'
touse = {}  # Initialize touse as an empty dictionary
counter = 0

# Populate touse with course titles containing the keyword
for key, value in course_dict.items():
    if keyword in key:
        touse[counter] = key
        counter += 1

# Print the results
print(touse)

In [ ]:
temp = df[df['course_title'].str.contains('Python')]
top6 = temp.sort_values(by='num_subscribers', ascending=False).head(6)


In [ ]:
index = course_index['How To Maximize Your Profits Trading Options']
scores = list(enumerate(cosine_sim_mat[index]))
sorted_score = sorted(scores, key=lambda x: x[1], reverse=True)


In [ ]:
# Assuming you already have the cosine similarity matrix and course_index

# Get the index of the course you want recommendations for
index = course_index['How To Maximize Your Profits Trading Options']

# Calculate the similarity scores
scores = list(enumerate(cosine_sim_mat[index]))

# Sort the scores
sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)




In [ ]:
# Select indices and values
sorted_indices = [i[0] for i in sorted_scores[1:]]  # Skip the first one (self-similarity)
sorted_values = [i[1] for i in sorted_scores[1:]]

# Now, create the DataFrame for recommended courses
recommended_result_df = df.iloc[sorted_indices]
recommended_result_df['Similarity_Score'] = np.array(sorted_values)

# Display the recommended courses
print(recommended_result_df[['course_title', 'Similarity_Score']])

In [ ]:
def recommend_course_based_on_keyword(keyword, numrec=10):
    # Find courses that contain the keyword in their titles
    matching_courses = df[df['Clean_title'].str.contains(keyword, case=False)]
    
    if matching_courses.empty:
        return "No courses found for this keyword."

    # Vectorize the Clean_title for the matching courses
    cv_mat = countvect.transform(matching_courses['Clean_title'])
    cosine_sim_mat = cosine_similarity(cv_mat)

    # Use the index of the first matching course
    index = 0  # We can take the first matching course for similarity comparison
    
    # Calculate similarity scores
    scores = list(enumerate(cosine_sim_mat[index]))
    
    # Sort the scores
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
    # Select indices and values
    sorted_indices = [i[0] for i in sorted_scores[1:]]  # Skip self-similarity
    sorted_values = [i[1] for i in sorted_scores[1:]]

    # Prepare recommended courses using the local indices from matching_courses
    recommended_result_df = matching_courses.iloc[sorted_indices].copy()  # Use .copy() to avoid the warning
    recommended_result_df.loc[:, 'Similarity_Score'] = np.array(sorted_values)  # Use .loc to set the score

    return recommended_result_df[['course_title', 'Similarity_Score', 'url', 'price', 'num_subscribers']].head(numrec)

# Now use the new function for "HTML"
html_recommendations = recommend_course_based_on_keyword('HTML', numrec=10)

# Display the results
print(html_recommendations)


In [ ]:
ans = recommend_course_based_on_keyword('graphics design', 10)
ans

In [ ]:
# confirm if a particular keyword is there
html_courses = df[df['course_title'].str.contains('HTML', case=False)]
html_courses